In [47]:
import math
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np

In [48]:
COMODOS = {
    "Quarto": (12, 30),
    "Banheiro": (3, 8),
    "Cozinha": (10, 15),
    "Sala de Estar": (30, 40),
    "Sala de Jantar": (15, 20),
    "Área de Serviço": (6, 10),
    "Sala de Jogos": (20, 30),
    "Closet": (3, 4),
}

MOBILIAS = {
    "Sala de Estar": [
        ("Sofá 2 lugares", 0.82, 1.72),
        ("Sofá 3 lugares", 0.82, 2.10),
        ("Poltrona", 0.70, 0.80),
        ("Mesa de Centro", 1, 0.60),
        ("Mesa Auxiliar", 0.43, 0.40),
    ],
    "Sala de Jantar": [
        ("Mesa 4 lugares", 0.90, 0.90),
        ("Mesa 6 lugares", 1.60, 0.90),
        ("Aparador", 0.90, 0.36),
        ("Cristaleira", 0.34, 0.84),
    ],
    "Quarto": [
        ("Cama de Casal", 1.44, 1.93),
        ("Cama de Solteiro", 0.94, 1.93),
        ("Mesa Auxiliar", 0.43, 0.40),
        ("Guarda-Roupa 2 portas", 0.40, 1),
        ("Guarda-Roupa 3 portas", 0.40, 1.70),
        ("Gaveteiro", 0.40, 0.87),
    ],
    "Cozinha": [
        ("Fogão 4 bocas", 0.58, 0.68),
        ("Fogão 5 bocas", 0.77, 0.68),
        ("Geladeira 1 porta", 0.62, 0.75),
        ("Geladeira 2 portas", 0.83, 0.79),
        ("Armário", 0.34, 1.68),
        ("Pia", 1, 0.50),
    ],
    "Banheiro": [
        ("Vaso Sanitário", 0.37, 0.64),
        ("Banheira", 0.71, 1.65),
        ("Pia com Armário", 0.70, 0.45),
        ("Box", 1.5, 2),
    ],
    "Área de Serviço": [("Máquina de Lavar", 0.60, 0.65), ("Tanque", 0.52, 0.53)],
    "Sala de Jogos": [
        ("Mesa de Sinuca", 1.37, 2.44),
        ("Mesa de Ping Pong", 1.52, 2.74),
        ("Sofá", 0.82, 2.10),
        ("Estante", 0.40, 1.80),
    ],
}

### Cromossomo

In [49]:
class Comodo:
    def __init__(self, tipo, andar, x, y, largura, comprimento):
        self.tipo = tipo
        self.andar = andar
        self.x = x
        self.y = y
        self.largura = largura
        self.comprimento = comprimento
        self.portas = random.randint(1, 3)
        self.janelas = random.randint(1, 3)
        self.mobilias = self.gerar_mobilias()

    def gerar_mobilias(self):
        mobilias = []
        espacos_ocupados = []

        for mob in MOBILIAS.get(self.tipo, []):
            mob_largura, mob_comprimento = mob[1], mob[2]

            if mob_largura <= self.largura and mob_comprimento <= self.comprimento:
                posicao = self.encontrar_posicao_nas_paredes(mob_largura, mob_comprimento, espacos_ocupados)
                
                if posicao:
                    mobilias.append(
                        {
                            "tipo": mob[0],
                            "largura": mob_largura,
                            "comprimento": mob_comprimento,
                            "posicao": posicao,
                        }
                    )
                    espacos_ocupados.append((posicao[0], posicao[1], mob_largura, mob_comprimento))

        return mobilias

    def encontrar_posicao_nas_paredes(self, mob_largura, mob_comprimento, espacos_ocupados):
        paredes = [
            (0, random.uniform(0, self.comprimento - mob_comprimento)),
            (self.largura - mob_largura, random.uniform(0, self.comprimento - mob_comprimento)),
            (random.uniform(0, self.largura - mob_largura), 0),
            (random.uniform(0, self.largura - mob_largura), self.comprimento - mob_comprimento)
        ]

        for posicao in paredes:
            if not self.verificar_sobreposicao(posicao, mob_largura, mob_comprimento, espacos_ocupados):
                return posicao

        return None

    def verificar_sobreposicao(self, posicao, mob_largura, mob_comprimento, espacos_ocupados):
        x_mob, y_mob = posicao
        for (x_ocupado, y_ocupado, largura_ocupada, comprimento_ocupado) in espacos_ocupados:
            if (
                x_mob < x_ocupado + largura_ocupada
                and x_mob + mob_largura > x_ocupado
                and y_mob < y_ocupado + comprimento_ocupado
                and y_mob + mob_comprimento > y_ocupado
            ):
                return True
        return False

In [50]:
class Planta:
    def __init__(self, area, orientacao):
        self.area = area
        self.largura_casa, self.comprimento_casa = self.gerar_dimensoes_planta()
        self.orientacao = orientacao
        self.grade = {
            0: [[0 for _ in range(self.largura_casa)] for _ in range(self.comprimento_casa)],
            1: [[0 for _ in range(self.largura_casa)] for _ in range(self.comprimento_casa)]
        }
        self.comodos = self.gerar_comodos_obrigatorios()
        self.fitness = self.calcular_fitness()

    def gerar_dimensoes_planta(self):
        proporcao_minima = 0.5
        proporcao_maxima = 2.0
    
        area_max = self.area
        proporcao = random.uniform(proporcao_minima, proporcao_maxima)
        
        comprimento_casa = int(math.sqrt(area_max * proporcao))
        largura_casa = int(area_max / comprimento_casa)
        
        return int(largura_casa), int(comprimento_casa)
    
    def gerar_dimensao_comodo(self, tipo):
        min_area, max_area = COMODOS[tipo]
        area = random.uniform(min_area, max_area)
        
        proporcao_minima = 0.5
        proporcao_maxima = 2.0
        
        while True:
            comprimento = random.uniform(1, math.sqrt(area))
            largura = area / comprimento
            
            proporcao = comprimento / largura
            
            if proporcao_minima <= proporcao <= proporcao_maxima:
                break
        
        return largura, comprimento
    
    def preencher_areas_vazias(self):
        """Preenche áreas vazias com pátios e corredores."""
        for andar in range(2):
            for i in range(self.comprimento_casa):
                for j in range(self.largura_casa):
                    if self.grade[andar][i][j] == 0:  # Se o espaço estiver vazio
                        # Decide aleatoriamente entre pátio ou corredor
                        tipo = "Pátio" if random.random() > 0.5 else "Corredor"
                        largura, comprimento = self.gerar_dimensao_comodo(tipo)
                        if i + comprimento <= self.comprimento_casa and j + largura <= self.largura_casa:
                            if not self.verificar_sobreposicao(andar, j, i, largura, comprimento):
                                # Marca a área como ocupada no grid
                                for m in range(int(i), int(i + comprimento)):
                                    for n in range(int(j), int(j + largura)):
                                        self.grade[andar][m][n] = 1
                                # Adiciona o pátio ou corredor como cômodo
                                comodo = Comodo(tipo, andar, j, i, largura, comprimento)
                                self.comodos.append(comodo)

    
    def gerar_comodos_obrigatorios(self):
        comodos = []
        comodos_obrigatorios = [
            ("Área de Serviço", 1),
            ("Banheiro", 4),
            ("Closet", 1),
            ("Cozinha", 1),
            ("Quarto", 3),
            ("Sala de Estar", 1),
            ("Sala de Jantar", 1),
            ("Sala de Jogos", 1),
        ]

        random.shuffle(comodos_obrigatorios)
        comodos = self.alocar_comodos(comodos_obrigatorios)

        return comodos

    def alocar_comodos(self, comodos_obrigatorios):
        comodos = []
        for tipo, quantidade in comodos_obrigatorios:
            for _ in range(quantidade):
                largura, comprimento = self.gerar_dimensao_comodo(tipo)
                localizacao = self.encontrar_melhor_localizacao(largura, comprimento)
                if localizacao:
                    x, y, andar = localizacao
                    comodo = Comodo(tipo, andar, x, y, largura, comprimento)
                    comodos.append(comodo)
        return comodos

    def encontrar_melhor_localizacao(self, largura, comprimento):
        for andar in range(2):
            for i in range(self.comprimento_casa):
                for j in range(self.largura_casa):
                    if i + comprimento <= self.comprimento_casa and j + largura <= self.largura_casa:
                        if not self.verificar_sobreposicao(andar, j, i, largura, comprimento):
                            for m in range(int(i), int(i + comprimento)):
                                for n in range(int(j), int(j + largura)):
                                    self.grade[andar][m][n] = 1
                            return j, i, andar
        return None

    def verificar_sobreposicao(self, andar, x, y, largura, comprimento):
        if y + comprimento > self.comprimento_casa or x + largura > self.largura_casa:
            return True
        
        for i in range(int(y), int(y + comprimento)):
            for j in range(int(x), int(x + largura)):
                if self.grade[andar][i][j] == 1:
                    return True
        return False

    def avaliar_area_utilizada(self):
        area_utilizada = sum(sum(row) for row in self.grade[0]) + sum(sum(row) for row in self.grade[1])
        total_area = self.largura_casa * self.comprimento_casa * 2
        return (area_utilizada / total_area) * 100
    
    def calcular_distancia(self, comodo1, comodo2):
        x1 = comodo1.x + comodo1.largura / 2
        y1 = comodo1.y + comodo1.comprimento / 2
        x2 = comodo2.x + comodo2.largura / 2
        y2 = comodo2.y + comodo2.comprimento / 2
        return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    def avaliar_posicao_escadas(self):
        pontuacao = 0
        escadas = [comodo for comodo in self.comodos if comodo.tipo == "Escada"]
        
        for escada in escadas:
            # A escada deve estar centralizada na planta
            centro_x = self.largura_casa / 2
            centro_y = self.comprimento_casa / 2
            distancia_centro = self.calcular_distancia_centro(escada, centro_x, centro_y)
            
            pontuacao += max(0, 10 - distancia_centro) * 10  # Quanto mais próximo do centro, maior a pontuação
        return pontuacao
    
    def avaliar_conexao_exterior(self):
        pontuacao = 0
        comodos_com_portas_externas = ["Sala de Estar", "Cozinha"]
        
        for comodo in self.comodos:
            if comodo.tipo in comodos_com_portas_externas:
                # Se o cômodo tem pelo menos uma porta na parede externa
                if self.esta_perto_das_paredes_externas(comodo):
                    pontuacao += 30  # Arbitrário, cada cômodo com porta externa recebe pontos
        return pontuacao


    def calcular_distancia_centro(self, comodo, centro_x, centro_y):
        comodo_centro_x = comodo.x + comodo.largura / 2
        comodo_centro_y = comodo.y + comodo.comprimento / 2
        return math.sqrt((comodo_centro_x - centro_x) ** 2 + (comodo_centro_y - centro_y) ** 2)
    
    def avaliar_iluminacao_natural(self):
        pontuacao = 0
        for comodo in self.comodos:
            # Se o cômodo está próximo às bordas da planta, recebe pontos
            if self.esta_perto_das_paredes_externas(comodo):
                pontuacao += 20  # Arbitrário, cada cômodo próximo à borda recebe pontos
        return pontuacao

    def esta_perto_das_paredes_externas(self, comodo):
        if (comodo.x == 0 or comodo.x + comodo.largura == self.largura_casa or
            comodo.y == 0 or comodo.y + comodo.comprimento == self.comprimento_casa):
            return True
        return False
    
    def avaliar_separacao_areas(self):
        pontuacao = 0
        areas_sociais = ["Sala de Estar", "Sala de Jantar", "Cozinha"]
        areas_privadas = ["Quarto", "Banheiro", "Closet"]
        
        for comodo in self.comodos:
            if comodo.tipo in areas_sociais:
                for outro_comodo in self.comodos:
                    if outro_comodo.tipo in areas_privadas:
                        dist = self.calcular_distancia(comodo, outro_comodo)
                        if dist < 3:
                            pontuacao -= 10
            else:
                pontuacao += 10
        return pontuacao
    
    def calcular_fitness(self):
        fitness = 0
        fitness += self.avaliar_area_utilizada()
        fitness += self.avaliar_separacao_areas()
        fitness += self.avaliar_iluminacao_natural()
        fitness += self.avaliar_conexao_exterior()
        fitness += self.avaliar_posicao_escadas()
        
        return fitness


In [51]:
def selecao(populacao):
    selecionados = random.sample(populacao, 2)
    return max(selecionados, key=lambda p: p.fitness)


def cruzamento(pai1, pai2):
    filho = Planta(pai1.largura_casa, pai1.comprimento_casa, pai1.orientacao)
    filho.comodos = (
        pai1.comodos[: len(pai1.comodos) // 2] + pai2.comodos[len(pai2.comodos) // 2 :]
    )
    filho.fitness = filho.calcular_fitness()
    return filho


def mutacao(planta):
    comodo = random.choice(planta.comodos)
    comodo.x = random.uniform(0, max(0.1, planta.largura_casa - comodo.largura))
    comodo.y = random.uniform(0, max(0.1, planta.comprimento_casa - comodo.comprimento))
    planta.fitness = planta.calcular_fitness()


def ciclo_evolutivo(
    geracoes, tamanho_populacao, area, orientacao
):
    populacao = [
        Planta(area, orientacao)
        for _ in range(tamanho_populacao)
    ]
    for geracao in range(geracoes):
        nova_populacao = []
        for _ in range(tamanho_populacao // 2):
            """ # Seleção
            pai1 = selecao(populacao)
            pai2 = selecao(populacao)
            
            # Cruzamento
            filho1 = cruzamento(pai1, pai2)
            filho2 = cruzamento(pai2, pai1)
            
            # Mutação com uma pequena probabilidade
            if random.random() < 0.1:
                mutacao(filho1)
            if random.random() < 0.1:
                mutacao(filho2)
            nova_populacao.extend([filho1, filho2]) """
            nova_populacao = populacao
        populacao = sorted(nova_populacao, key=lambda p: p.fitness, reverse=True)[
            :tamanho_populacao
        ]
    return populacao

In [ ]:
area = 150
orientacao = "norte"

melhor_planta = ciclo_evolutivo(
    geracoes=1000,
    tamanho_populacao=1000,
    area=area,
    orientacao=orientacao,
)

In [39]:
def desenhar_planta(planta):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    axes = [ax1, ax2]
    titles = ["Planta do Térreo", "Planta do Primeiro Andar"]

    for ax, title in zip(axes, titles):
        ax.set_title(title)
        ax.set_xlim(0, planta.largura_casa)
        ax.set_ylim(0, planta.comprimento_casa)
        ax.set_aspect("equal")

    cores = plt.cm.Set3(np.linspace(0, 1, len(COMODOS)))
    cor_mapa = dict(zip(COMODOS.keys(), cores))

    for comodo in planta.comodos:
        ax = axes[comodo.andar]
        retangulo = Rectangle(
            (comodo.x, comodo.y),
            comodo.largura,
            comodo.comprimento,
            fill=True,
            facecolor=cor_mapa[comodo.tipo],
            edgecolor="black",
            linewidth=2,
        )
        ax.add_patch(retangulo)
        ax.text(
            comodo.x + comodo.largura / 2,
            comodo.y + comodo.comprimento / 2,
            comodo.tipo,
            ha="center",
            va="center",
            wrap=True,
        )

        # Desenhar portas
        for _ in range(comodo.portas):
            porta_x = comodo.x + random.uniform(0, comodo.largura)
            porta_y = (
                comodo.y
                if random.choice([True, False])
                else comodo.y + comodo.comprimento
            )
            porta = Rectangle(
                (porta_x, porta_y),
                0.8,
                0.1,
                fill=True,
                facecolor="brown",
                edgecolor="black",
            )
            ax.add_patch(porta)

        # Desenhar janelas
        for _ in range(comodo.janelas):
            janela_x = comodo.x + random.uniform(0, comodo.largura)
            janela_y = (
                comodo.y
                if random.choice([True, False])
                else comodo.y + comodo.comprimento
            )
            janela = Rectangle(
                (janela_x, janela_y),
                1,
                0.1,
                fill=True,
                facecolor="lightblue",
                edgecolor="black",
            )
            ax.add_patch(janela)

        # Desenhar móveis
        for mobilia in comodo.mobilias:
            mob_x = comodo.x + mobilia["posicao"][0]
            mob_y = comodo.y + mobilia["posicao"][1]
            movel = Rectangle(
                (mob_x, mob_y),
                mobilia["largura"],
                mobilia["comprimento"],
                fill=True,
                facecolor="gray",
                edgecolor="black",
                alpha=0.5,
            )
            ax.add_patch(movel)

    # Adicionar orientação
    orientacao_texto = f"N\n^\n|\n \n{planta.orientacao}"
    for ax in axes:
        ax.text(
            1.02, 0.5, orientacao_texto, transform=ax.transAxes, va="center", ha="left"
        )

    plt.tight_layout()
    plt.show()

In [ ]:
# Desenhar a planta
desenhar_planta(melhor_planta[0])

# Exibir informações sobre a planta
print("\nInformações da planta gerada:")
# for comodo in melhor_planta.comodos:
#     print(f"Cômodo: {comodo.tipo}, Andar: {comodo.andar}")
#     print(f"Posição: ({comodo.x:.2f}, {comodo.y:.2f})")
#     print(f"Tamanho: {comodo.largura:.2f}m x {comodo.comprimento:.2f}m")
#     print(f"Portas: {comodo.portas}, Janelas: {comodo.janelas}")
#     for mob in comodo.mobilias:
#         print(
#             f"Mobília: {mob['tipo']}, Tamanho: {mob['largura']:.2f}m x {mob['comprimento']:.2f}m, Posição: {mob['posicao']}"
#         )
#     print()

# print(f"Fitness da planta: {melhor_planta.fitness}")